In [1]:
from extract import extract_collection
from dotenv import load_dotenv
import os
import logging
import json
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

[{'_id': ObjectId('6705472f45d606c54bf996cb'), 'user_id': '7c50a381-1670-4173-8ff8-8aa18e6f3f78', 'name': 'Kelly Becker', 'age': None, 'email': '', 'address': {'street': '912 Diana Manors', 'city': 'Mcphersonbury', 'zip_code': '08581'}}, {'_id': ObjectId('6705472f45d606c54bf996cc'), 'user_id': 'f9473811-fca7-4f57-bab0-71524be069b7', 'name': 'John Lyons', 'age': None, 'email': None, 'address': {'street': '86294 Tanner Summit Apt. 677', 'city': 'Pricechester', 'zip_code': '16697'}}, {'_id': ObjectId('6705472f45d606c54bf996cd'), 'user_id': '6cb9f67c-9f24-4e83-943c-116da2d6b710', 'name': 'Deborah Jones', 'age': None, 'email': 'emily49@example.com', 'address': {'street': '8885 Becky Pines Apt. 060', 'city': 'South Jessica', 'zip_code': '60590'}}, {'_id': ObjectId('6705472f45d606c54bf996ce'), 'user_id': '697e3f8a-4a8c-4303-978b-9fcf922318ef', 'name': 'Megan Mills', 'age': 'unknown', 'email': '', 'address': {'street': '4961 Karl Pines Suite 657', 'city': 'New Pamelaberg', 'zip_code': None}}, 

In [2]:
load_dotenv()

logging.basicConfig(
    filename="./logs/extract_log.log",
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

MONGO_URL = os.getenv("MONGODB_URI")

In [3]:
db="product_transactions"
collection="users"
users_data = extract_collection(MONGO_URL,db,collection,projection={"_id": 0})
users_data

[{'user_id': '7c50a381-1670-4173-8ff8-8aa18e6f3f78',
  'name': 'Kelly Becker',
  'age': None,
  'email': '',
  'address': {'street': '912 Diana Manors',
   'city': 'Mcphersonbury',
   'zip_code': '08581'}},
 {'user_id': 'f9473811-fca7-4f57-bab0-71524be069b7',
  'name': 'John Lyons',
  'age': None,
  'email': None,
  'address': {'street': '86294 Tanner Summit Apt. 677',
   'city': 'Pricechester',
   'zip_code': '16697'}},
 {'user_id': '6cb9f67c-9f24-4e83-943c-116da2d6b710',
  'name': 'Deborah Jones',
  'age': None,
  'email': 'emily49@example.com',
  'address': {'street': '8885 Becky Pines Apt. 060',
   'city': 'South Jessica',
   'zip_code': '60590'}},
 {'user_id': '697e3f8a-4a8c-4303-978b-9fcf922318ef',
  'name': 'Megan Mills',
  'age': 'unknown',
  'email': '',
  'address': {'street': '4961 Karl Pines Suite 657',
   'city': 'New Pamelaberg',
   'zip_code': None}},
 {'user_id': 'd39bf703-c661-4ec5-9e87-461175f5dd71',
  'name': 'Joseph Williams',
  'age': None,
  'email': '',
  'addres

In [4]:
sc = SparkContext(master="local[*]")


24/10/11 22:33:53 WARN Utils: Your hostname, gabriel-BOD-WXX9 resolves to a loopback address: 127.0.1.1; using 192.168.136.211 instead (on interface wlp0s20f3)
24/10/11 22:33:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/11 22:33:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark = SparkSession.builder.appName("users").getOrCreate()

In [11]:
df = sc.parallelize(users_data).map(lambda x: json.dumps(x))
df = spark.read.json(df)

Py4JJavaError: An error occurred while calling o5053.load.
: java.lang.ClassCastException: class java.util.HashMap cannot be cast to class java.lang.String (java.util.HashMap and java.lang.String are in module java.base of loader 'bootstrap')
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(DataSource.scala:551)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:404)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:229)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:211)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:211)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.inte

In [8]:
import pyspark.sql.functions as F

In [7]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- zip_code: string (nullable = true)
 |-- age: string (nullable = true)
 |-- email: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)

